In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data14168


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

res_cbt_kfold7_0.6477160698084005.csv  res_xgb_kfold6_0.6876512726718488.csv
res_cbt_kfold7_0.6898339248774186.csv  res_xgb_kfold7_0.699708384479906.csv
res_cbt_kfold7_0.6975520165312812.csv  res_xgb_kfold7_0.7046168234288157.csv
res_cbt_kfold7_0.7171472830358574.csv  res_xgb_kfold7_0.7306788309565827.csv
res_cbt_kfold7_0.7236086940573763.csv


In [3]:
!pip install xgboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 142.8MB 217kB/s ta 0:00:011 6% |██                              | 8.7MB 3.9MB/s eta 0:00:35    25% |████████▏                       | 36.5MB 2.9MB/s eta 0:00:37


In [4]:
! pip install catboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 62.8MB 523kB/s eta 0:00:01
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='mirrors.ustc.edu.cn', port=443): Read timed out. (read timeout=15)",)': /pypi/web/simple/plotly/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='mirrors.ustc.edu.cn', port=443): Read timed out. (read timeout=15)",)': /pypi/web/simple/plotly/
    100% |████████████████████████████████| 7.2MB 2.3MB/s ta 0:00:011
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/aistudio/.cache/pip/wheels/5b/e5/21/797e8ecf814b6ab2399ab254682b20dda1d244183a0e372010
Successfully built retrying


In [5]:
!pip install imblearn

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 174kB 2.2MB/s ta 0:00:01


In [6]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
import xgboost as xgb
import catboost as cbt
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import pandas as pd
from imblearn.over_sampling import SMOTE
# 处理样本不均衡

In [7]:
data_path = '/home/aistudio/data/data14168/train.csv'
predict_data_path = '/home/aistudio/data/data14168/test.csv'
label_path = '/home/aistudio/data/data14168/train_target.csv'

In [8]:
# 这个就是进行缺失值处理
def impute_NA_with_avg(data,strategy='mean',NA_col=[]):
    """
    replacing the NA with mean/median/most frequent values of that variable. 
    Note it should only be performed over training set and then propagated to test set.
    """
    
    data_copy = data.copy(deep=True)
    for i in NA_col:
        if data_copy[i].isnull().sum()>0:
            if strategy=='mean':
                data_copy[i+'_impute_mean'] = data_copy[i].fillna(data[i].mean())
            elif strategy=='median':
                data_copy[i+'_impute_median'] = data_copy[i].fillna(data[i].median())
            elif strategy=='mode':
                data_copy[i+'_impute_mode'] = data_copy[i].fillna(data[i].mode()[0])
        else:
            warn("Column %s has no missing" % i)
    return data_copy  

In [9]:
df = pd.read_csv(data_path)
df_label  = pd.read_csv(label_path)
df_predict  = pd.read_csv(predict_data_path)

In [10]:
df

,id,certId,loanProduct,gender,age,dist,edu,job,lmt,basicLevel,...,highestEdu,linkRela,setupHour,weekday,ncloseCreditCard,unpayIndvLoan,unpayOtherLoan,unpayNormalLoan,5yearBadloan,isNew
0,1,640582,1,2,27,640500,0,8,5.963,3,...,-999,0,9,6,1,0,1,0,0,1
1,2,621023,2,2,25,640600,0,3,6.467,3,...,99,11,6,5,1,0,1,0,0,1
2,3,641283,1,2,37,641200,70,5,0.596,1,...,70,0,1,6,0,0,0,0,0,1
3,4,340821,3,2,29,340821,0,4,6.300,3,...,-999,-999,13,6,1,0,1,0,0,1
4,5,732628,2,2,22,732628,0,2,6.700,2,...,99,11,8,3,1,0,1,0,0,1
5,6,610221,3,2,27,610221,0,4,5.867,3,...,-999,-999,11,3,1,0,1,1,0,1
6,7,629004,1,1,31,620100,0,3,12.630,1,...,-999,0,17,5,1,1,0,1,0,1
7,8,822223,1,2,32,822223,0,3,1.096,1,...,99,0,14,6,1,1,1,0,0,1
8,9,632503,1,2,27,640700,0,4,0.830,2,...,-999,0,2,6,0,0,1,0,0,1
9,10,722323,2,2,37,722300,0,3,7.667,1,...,-999,12,17,5,0,0,1,0,0,1


In [11]:
column_headers = list( df.columns.values )
column_headers

['id',
 'certId',
 'loanProduct',
 'gender',
 'age',
 'dist',
 'edu',
 'job',
 'lmt',
 'basicLevel',
 'x_0',
 'x_1',
 'x_2',
 'x_3',
 'x_4',
 'x_5',
 'x_6',
 'x_7',
 'x_8',
 'x_9',
 'x_10',
 'x_11',
 'x_12',
 'x_13',
 'x_14',
 'x_15',
 'x_16',
 'x_17',
 'x_18',
 'x_19',
 'x_20',
 'x_21',
 'x_22',
 'x_23',
 'x_24',
 'x_25',
 'x_26',
 'x_27',
 'x_28',
 'x_29',
 'x_30',
 'x_31',
 'x_32',
 'x_33',
 'x_34',
 'x_35',
 'x_36',
 'x_37',
 'x_38',
 'x_39',
 'x_40',
 'x_41',
 'x_42',
 'x_43',
 'x_44',
 'x_45',
 'x_46',
 'x_47',
 'x_48',
 'x_49',
 'x_50',
 'x_51',
 'x_52',
 'x_53',
 'x_54',
 'x_55',
 'x_56',
 'x_57',
 'x_58',
 'x_59',
 'x_60',
 'x_61',
 'x_62',
 'x_63',
 'x_64',
 'x_65',
 'x_66',
 'x_67',
 'x_68',
 'x_69',
 'x_70',
 'x_71',
 'x_72',
 'x_73',
 'x_74',
 'x_75',
 'x_76',
 'x_77',
 'x_78',
 'certValidBegin',
 'certValidStop',
 'bankCard',
 'ethnic',
 'residentAddr',
 'highestEdu',
 'linkRela',
 'setupHour',
 'weekday',
 'ncloseCreditCard',
 'unpayIndvLoan',
 'unpayOtherLoan',
 'unpayN

In [12]:
train_data = pd.merge(df, df_label, how='left', on=['id'])

In [13]:
train_data.dropna(how='all')
# 这个是用于滤除缺失值的，how=all就是只丢弃全部为缺失值的行

,id,certId,loanProduct,gender,age,dist,edu,job,lmt,basicLevel,...,linkRela,setupHour,weekday,ncloseCreditCard,unpayIndvLoan,unpayOtherLoan,unpayNormalLoan,5yearBadloan,isNew,target
0,1,640582,1,2,27,640500,0,8,5.963,3,...,0,9,6,1,0,1,0,0,1,0
1,2,621023,2,2,25,640600,0,3,6.467,3,...,11,6,5,1,0,1,0,0,1,0
2,3,641283,1,2,37,641200,70,5,0.596,1,...,0,1,6,0,0,0,0,0,1,0
3,4,340821,3,2,29,340821,0,4,6.300,3,...,-999,13,6,1,0,1,0,0,1,0
4,5,732628,2,2,22,732628,0,2,6.700,2,...,11,8,3,1,0,1,0,0,1,0
5,6,610221,3,2,27,610221,0,4,5.867,3,...,-999,11,3,1,0,1,1,0,1,0
6,7,629004,1,1,31,620100,0,3,12.630,1,...,0,17,5,1,1,0,1,0,1,0
7,8,822223,1,2,32,822223,0,3,1.096,1,...,0,14,6,1,1,1,0,0,1,0
8,9,632503,1,2,27,640700,0,4,0.830,2,...,0,2,6,0,0,1,0,0,1,0
9,10,722323,2,2,37,722300,0,3,7.667,1,...,12,17,5,0,0,1,0,0,1,0


In [14]:
# 缺失值进行填充
train_data = impute_NA_with_avg(train_data, column_headers) 

df_predict = impute_NA_with_avg(df_predict, column_headers) 

In [15]:
train_data.fillna(0, inplace = True) 
df_predict.fillna(0, inplace = True) 

In [16]:
train_data
groupby_data_orginal=train_data.groupby('target').count()

,id,certId,loanProduct,gender,age,dist,edu,job,lmt,basicLevel,...,linkRela,setupHour,weekday,ncloseCreditCard,unpayIndvLoan,unpayOtherLoan,unpayNormalLoan,5yearBadloan,isNew,target
0,1,640582,1,2,27,640500,0,8,5.963,3,...,0,9,6,1,0,1,0,0,1,0
1,2,621023,2,2,25,640600,0,3,6.467,3,...,11,6,5,1,0,1,0,0,1,0
2,3,641283,1,2,37,641200,70,5,0.596,1,...,0,1,6,0,0,0,0,0,1,0
3,4,340821,3,2,29,340821,0,4,6.300,3,...,-999,13,6,1,0,1,0,0,1,0
4,5,732628,2,2,22,732628,0,2,6.700,2,...,11,8,3,1,0,1,0,0,1,0
5,6,610221,3,2,27,610221,0,4,5.867,3,...,-999,11,3,1,0,1,1,0,1,0
6,7,629004,1,1,31,620100,0,3,12.630,1,...,0,17,5,1,1,0,1,0,1,0
7,8,822223,1,2,32,822223,0,3,1.096,1,...,0,14,6,1,1,1,0,0,1,0
8,9,632503,1,2,27,640700,0,4,0.830,2,...,0,2,6,0,0,1,0,0,1,0
9,10,722323,2,2,37,722300,0,3,7.667,1,...,12,17,5,0,0,1,0,0,1,0


In [17]:
column_headers = list( train_data.columns.values )
print(column_headers)

['id', 'certId', 'loanProduct', 'gender', 'age', 'dist', 'edu', 'job', 'lmt', 'basicLevel', 'x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78', 'certValidBegin', 'certValidStop', 'bankCard', 'ethnic', 'residentAddr', 'highestEdu', 'linkRela', 'setupHour', 'weekday', 'ncloseCreditCard', 'unpayIndvLoan', 'unpayOtherLoan', 'unpayNormalLoan', '5yearBadloan', 'isNew', 'target']


In [18]:
x_columns = [x for x in train_data.columns if x not in ["target", "id"]]

In [18]:
X = train_data[x_columns]
y = train_data["target"] 
X_predict = df_predict[x_columns]
X_train, y_train = X, y

In [ ]:
model_smote=SMOTE()    #建立smote模型对象
x_smote_resampled,y_smote_resampled=model_smote.fit_sample(X,y)
x_smote_resampled=pd.DataFrame(x_smote_resampled,columns=x_columns)
y_smote_resampled=pd.DataFrame(y_smote_resampled,columns=['label'])
smote_resampled=pd.concat([x_smote_resampled,y_smote_resampled],axis=1)
groupby_data_smote=smote_resampled.groupby('label').count()

In [21]:
# one-hot处理
train_data = X_train
test_data = X_predict
# 选择要做onehot的列['id', 'certId', 'loanProduct', 'gender', 'age', 'dist', 'edu', 'job', 'lmt', 'basicLevel', 'x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78', 'certValidBegin', 'certValidStop', 'bankCard', 'ethnic', 'residentAddr', 'highestEdu', 'linkRela', 'setupHour', 'weekday', 'ncloseCreditCard', 'unpayIndvLoan', 'unpayOtherLoan', 'unpayNormalLoan', '5yearBadloan', 'isNew', 'target']
# ["gender", "edu", "job", 'x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78']
dummy_fea = ["gender", "edu", "job", 'x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78']
train_test_data = pd.concat([train_data,test_data],axis=0,ignore_index = True)
train_test_data = train_test_data.fillna(0)
dummy_df = pd.get_dummies(train_test_data.loc[:,dummy_fea])
train_test_data = pd.concat([train_test_data,dummy_df],axis=1)
train_test_data = train_test_data.drop(dummy_fea,axis=1)
train_train = train_test_data.iloc[:train_data.shape[0],:]
test_test = train_test_data.iloc[train_data.shape[0]:,:]

In [19]:
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=829111)
max_auc = 0
for train_index, test_index in kf.split(X_train):
    print ( ">>>", train_index )
    xgboost = xgb.XGBClassifier(max_depth=5, n_estimators=800, learning_rate=0.01,)
    xgboost_model = xgboost.fit(X_train.iloc[train_index], y_train.iloc[train_index]) 

    y_pred = xgboost_model.predict(X_train.iloc[test_index]) 
    y_predprob = xgboost_model.predict_proba(X_train.iloc[test_index])[:, 1] 
    print (y_predprob)

    print("Accuracy : %.4g" % metrics.accuracy_score(y_train.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 

    if auc > max_auc:
        max_auc = auc
        print ("auc>>>>>>", auc)
        y_pp = xgboost_model.predict_proba(X_predict)[:, 1] 
        # print (y_pp)
        c ={ "target" : y_pp }
        data_lable = DataFrame(c)#将字典转换成为数据框
        # print ( data_lable )
        id_list = df_predict["id"].tolist()
        # print ( id_list )


        d ={ "id" : id_list, "target" : y_pp  }
        res = DataFrame(d)#将字典转换成为数据框
        print (">>>>", res)
        csv_file = '/home/aistudio/work/res_xgb_kfold' + str(n_splits) + '_'  + str(auc) + '.csv'
        res.to_csv(csv_file,index = None) 

>>> [     0      1      2 ... 132026 132027 132028]
[0.00179888 0.01590867 0.01695535 ... 0.00740959 0.00651568 0.0068048 ]
Accuracy : 0.9936
AUC Score (Train): 0.675643
auc>>>>>> 0.6756425333716632
>>>>            id    target
0      132030  0.007642
1      132031  0.018497
2      132032  0.004752
3      132033  0.003216
4      132034  0.023872
5      132035  0.008471
6      132036  0.002453
7      132037  0.020880
8      132038  0.017140
9      132039  0.011556
10     132040  0.003438
11     132041  0.036676
12     132042  0.002448
13     132043  0.004546
14     132044  0.011962
15     132045  0.003805
16     132046  0.019533
17     132047  0.027516
18     132048  0.013035
19     132049  0.008666
20     132050  0.002940
21     132051  0.004457
22     132052  0.013498
23     132053  0.003864
24     132054  0.003823
25     132055  0.005726
26     132056  0.002469
27     132057  0.002635
28     132058  0.002366
29     132059  0.013868
...       ...       ...
23531  155561  0.002656
2353

In [20]:
# n_splits = 7
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=829128)
# max_auc = 0
# for train_index, test_index in kf.split(X_train):
#     print ( ">>>", train_index )
#     catboost = cbt.CatBoostClassifier()
#     # 注意这要记得加括号，不然会报错'DataFrame' object has no attribute '_init_params'
#     catboost_model = catboost.fit(X_train.iloc[train_index], y_train.iloc[train_index]) 


#     y_pred = catboost_model.predict(X_train.iloc[test_index]) 
#     y_predprob = catboost_model.predict_proba(X_train.iloc[test_index])[:, 1] 
#     print (y_predprob)

#     print("Accuracy : %.4g" % metrics.accuracy_score(y_train.iloc[test_index].values, y_pred))  
#     auc = metrics.roc_auc_score(y_train.iloc[test_index], y_predprob)
#     print("AUC Score (Train): %f" % auc) 

#     if auc > max_auc:
#         max_auc = auc
#         print ("auc>>>>>>", auc)
#         y_pp = catboost_model.predict_proba(X_predict)[:, 1] 
#         # print (y_pp)
#         c ={ "target" : y_pp }
#         data_lable = DataFrame(c)#将字典转换成为数据框
#         # print ( data_lable )
#         id_list = df_predict["id"].tolist()
#         # print ( id_list )


#         d ={ "id" : id_list, "target" : y_pp  }
#         res = DataFrame(d)#将字典转换成为数据框
#         print (">>>>", res)
#         csv_file = '/home/aistudio/work/res_cbt_kfold' + str(n_splits) + '_'  + str(auc) + '.csv'
#         res.to_csv(csv_file,index = None)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
# 做一个网格搜索试一下